# Add Folders to Path

In [1]:
%%time
import sys, os
# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
# add Algorithm folder to current working path in order to access the functions inside the folder ´Algorithms´
sys.path.append(parent_directory+"/General_Functions")

CPU times: user 195 µs, sys: 126 µs, total: 321 µs
Wall time: 211 µs


# Agulhas

## Import Data

In [2]:
%%time
import scipy.io as sio
import numpy as np

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../Data/TensorFieldData.mat')

x = mat_file['x1_g']
y = mat_file['x2_g']
C11 = mat_file['C11']
C12 = mat_file['C12']
C22 = mat_file['C22']
lam2 = mat_file['lam2']
C11_x = mat_file['C11x1']
C11_y = mat_file['C11x2']
C12_x = mat_file['C12x1']
C12_y = mat_file['C12x2']
C22_x = mat_file['C22x1']
C22_y = mat_file['C22x2']

Ncores = 18

X_domain, Y_domain = np.meshgrid(x, y)

CPU times: user 227 ms, sys: 41.9 ms, total: 269 ms
Wall time: 269 ms


In [3]:
%%time
import numpy as np
from ipynb.fs.defs.init_level_set import _init_level_set
from ipynb.fs.defs.gradient_C import _gradient_C
from ipynb.fs.defs.geodesic_equation import _geodesic_equation
from ipynb.fs.defs.RK4_geodesic import _RK4_geodesic
from ipynb.fs.defs.phi_prime import _phi_prime

lambda_range = np.arange(0.9, 1.1+0.02, 0.02)

#grad_C11, grad_C12, grad_C22 = _gradient_C(X_domain, Y_domain, C)
x0lam, y0lam, phi0lam = _init_level_set(X_domain, Y_domain, C11, C12, lambda_range)

from scipy.interpolate import RectBivariateSpline as RBS
        
# Interpolant for components of Cauchy-Green strain tensor
interp_C11 = RBS(Y_domain[:,0], X_domain[0,:], C11, kx = 1, ky = 1)
interp_C12 = RBS(Y_domain[:,0], X_domain[0,:], C12, kx = 1, ky = 1)
interp_C22 = RBS(Y_domain[:,0], X_domain[0,:], C22, kx = 1, ky = 1)

interp_phi_prime = _phi_prime(X_domain, Y_domain, C11, C12, C22, C11_x, C11_y, C12_x, C12_y, C22_x, C22_y, Ncores)

  0%|          | 0/801 [00:00<?, ?it/s]

CPU times: user 5.35 s, sys: 1 s, total: 6.35 s
Wall time: 1min 26s


In [4]:
print(interp_phi_prime(np.array([-30, -2, 0])))

[-4.35421621]


In [ ]:
x_phi = np.array([-2.67, -29.27, 0])

ds = 0.01

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16, 10))
ax = plt.axes()

for i in tqdm(range(len(x0lam))):
    
    x_phi = np.array([x0lam[i], y0lam[i], phi0lam[i]])

    defined_domain = np.isfinite(C11).astype(int)

    trajectory = []

    counter = 0

    while counter < 2000:
    
        trajectory.append(x_phi)
    
        x_phi = _RK4_geodesic(X_domain, Y_domain, defined_domain, ds, x_phi, interp_phi_prime, interp_C11, interp_C22, interp_C12)[0]
        counter += 1
    
        if x_phi is None:
            break
    trajectory = np.array(trajectory)
    ax = plt.axes()
    ax.plot(trajectory[:,0], trajectory[:,1])
    
    trajectory = np.array(trajectory)
plt.show()

  0%|          | 0/98924 [00:00<?, ?it/s]